# ExLlamaV2

[ExLlamav2](https://github.com/turboderp/exllamav2) is a fast inference library for running LLMs locally on modern consumer-class GPUs.

It supports inference for GPTQ & EXL2 quantized models, which can be accessed on [Hugging Face](https://huggingface.co/TheBloke).

This notebook goes over how to run `exllamav2` within LangChain.

Additional information:
[ExLlamav2 examples](https://github.com/turboderp/exllamav2/tree/master/examples)


## Installation

Refer to the official [doc](https://github.com/turboderp/exllamav2)
For this notebook, the requirements are :
- python 3.11
- langchain 0.1.7
- CUDA: 12.1.0 (see bellow)
- torch==2.1.1+cu121
- exllamav2 (0.0.12+cu121)

If you want to install the same exllamav2 version :
```shell
pip install https://github.com/turboderp/exllamav2/releases/download/v0.0.12/exllamav2-0.0.12+cu121-cp311-cp311-linux_x86_64.whl
```

if you use conda, the dependencies are :
```
  - conda-forge::ninja
  - nvidia/label/cuda-12.1.0::cuda
  - conda-forge::ffmpeg
  - conda-forge::gxx=11.4
```

## Usage

You don't need an `API_TOKEN` as you will run the LLM locally.

It is worth understanding which models are suitable to be used on the desired machine.

[TheBloke's](https://huggingface.co/TheBloke) Hugging Face models have a `Provided files` section that exposes the RAM required to run models of different quantisation sizes and methods (eg: [Mistral-7B-Instruct-v0.2-GPTQ](https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GPTQ)).


In [3]:
!pip install -U langchain huggingface_hub exllamav2 langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/langchain-ai/langchain.git

Cloning into 'langchain'...
remote: Enumerating objects: 234693, done.
remote: Counting objects: 100% (829/829), done.
remote: Compressing objects: 100% (342/342), done.
remote: Total 234693 (delta 613), reused 492 (delta 487), pack-reused 233864 (from 2)
Receiving objects: 100% (234693/234693), 403.70 MiB | 31.50 MiB/s, done.
Resolving deltas: 100% (177404/177404), done.
Updating files: 100% (7411/7411), done.


In [6]:
import os

from huggingface_hub import snapshot_download
from langchain_community.llms.exllamav2 import ExLlamaV2
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
#from libs.langchain.langchain.chains.llm import LLMChain

In [7]:
# function to download the gptq model
def download_GPTQ_model(model_name: str, models_dir: str = "./models/") -> str:
    """Download the model from hugging face repository.

    Params:
    model_name: str: the model name to download (repository name). Example: "TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ"
    """
    # Split the model name and create a directory name. Example: "TheBloke/CapybaraHermes-2.5-Mistral-7B-GPTQ" -> "TheBloke_CapybaraHermes-2.5-Mistral-7B-GPTQ"

    if not os.path.exists(models_dir):
        os.makedirs(models_dir)

    _model_name = model_name.split("/")
    _model_name = "_".join(_model_name)
    model_path = os.path.join(models_dir, _model_name)
    if _model_name not in os.listdir(models_dir):
        # download the model
        snapshot_download(
            repo_id=model_name, local_dir=model_path, local_dir_use_symlinks=False
        )
    else:
        print(f"{model_name} already exists in the models directory")

    return model_path

In [8]:
from exllamav2.generator import (
    ExLlamaV2Sampler,
)

settings = ExLlamaV2Sampler.Settings()
settings.temperature = 0.85
settings.top_k = 50
settings.top_p = 0.8
settings.token_repetition_penalty = 1.05

model_path = download_GPTQ_model("TheBloke/Mistral-7B-Instruct-v0.2-GPTQ")

callbacks = [StreamingStdOutCallbackHandler()]

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

# Verbose is required to pass to the callback manager
llm = ExLlamaV2(
    model_path=model_path,
    callbacks=callbacks,
    verbose=True,
    settings=settings,
    streaming=True,
    max_new_tokens=150,
)
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What Football team won the UEFA Champions League in the year the iphone 6s was released?"

output = llm_chain.invoke({"question": question})
print(output)

Output()

Loading exllamav2_ext extension (JIT)...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

{'token_repetition_penalty': 1.05, 'token_repetition_range': -1, 'token_repetition_decay': 0, 'token_frequency_penalty': 0.0, 'token_presence_penalty': 0.0, 'temperature': 0.85, 'smoothing_factor': 0.0, 'min_temp': 0, 'max_temp': 0.0, 'temp_exponent': 1.0, 'top_k': 50, 'top_p': 0.8, 'top_a': 0.0, 'min_p': 0, 'tfs': 0, 'typical': 0, 'skew': 0, 'temperature_last': False, 'mirostat': False, 'mirostat_tau': 1.5, 'mirostat_eta': 0.1, 'mirostat_mu': None, 'token_bias': None, 'cfg_scale': None, 'post_sampling_hooks': [], 'dry_allowed_length': 2, 'dry_base': 1.75, 'dry_multiplier': 0.0, 'dry_sequence_breakers': None, 'dry_range': 0, 'dry_max_ngram': 20, 'ngram_trie': None, 'ngram_index': 0, 'ngram_history': deque([]), 'xtc_probability': 0.0, 'xtc_threshold': 0.1, 'xtc_ignore_tokens': None}
stop_sequences []


<ipython-input-8-bad80366ee82>:30: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


 We know that the iPhone 6s was released on September 25, 2015. The UEFA Champions League final match is usually held in May of each year.

Let's see which team won the UEFA Champions League in May 2015 or earlier:

1. Barcelona (2014-15)
2. Real Madrid (2013-14)
3. Bayern Munich (2012-13)
4. Chelsea (2011-12)
5. Inter Milan (2009-10)

None of these teams won the UEFA Champions League in May 2015 or{'question': 'What Football team won the UEFA Champions League in the year the iphone 6s was released?', 'text': " We know that the iPhone 6s was released on September 25, 2015. The UEFA Champions League final match is usually held in May of each year.\n\nLet's see which team won the UEFA Champions League in May 2015 or earlier:\n\n1. Barcelona (2014-15)\n2. Real Madrid (2013-14)\n3. Bayern Munich (2012-13)\n4. Chelsea (2011-12)\n5. Inter Milan (2009-10)\n\nNone of these teams won the UEFA Champions League in May 2015 or"}


In [9]:
import gc

import torch

torch.cuda.empty_cache()
gc.collect()
!nvidia-smi

Thu Apr 10 03:35:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P0             26W /   70W |    8278MiB /  15360MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----